In [1]:
# Intallation packages before use Jupyter

!pip install pandas
!pip install -U scikit-learn
!pip install keras
!pip install matplotlib
!pip install kaleido
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 80.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 48.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 32.7 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 65.2 MB/s eta 0:00:00:00:0100:01


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
# from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# be able to save images on server
matplotlib.use('Agg')

# !pip install -U kaleido # w Google Colab wymagany Runtime restart po instalacji (Runtime -> Restart Runtime)
import kaleido #required
# kaleido.__version__ #0.2.1

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2024-06-10 19:11:00.847557: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.16.1
Num GPUs Available:  1


In [6]:
# Zip
# importing the "tarfile" module
import tarfile # type: ignore

# open file
file = tarfile.open('1_150x9.tar.gz')

# extracting a specific file
file.extractall(path='./1_150x9/')

file.close()

In [7]:
# Data generator
def data():
    # Timestep 
    timestepsPerSample = 20

    # Timesteps in input data
    timestepsPerSampleWholeData = 150

    # Replace 'your_file.csv' with the actual file path
    file_features = './1_150x9/1_150x9f.csv'
    file_labels = './1_150x9/1_150x9l.csv'

    # Read the .csv file and create an array
    data_strings = np.genfromtxt(file_features, delimiter=';')
    labels_strings = np.genfromtxt(file_labels,delimiter=';')

    # Wycinanie wybranych kolumn
    # data_s = data_strings[:,[0,2,3,4,5,6,7,8]]
    data_s = data_strings[:,[0,4,5,8]]
    num_features = data_s.shape[1]
    # print(data_strings[:3])
    # print(data_s[:3])
    
    # Convert from strings to float and int
    X = data_s.astype(float).reshape((-1,timestepsPerSampleWholeData,num_features))
    Y = labels_strings.astype(float).reshape((-1,3))
    # print(X.shape)
    # print(Y.shape)

    # Modification from imported to new size X[samples,timesteps]
    X_mod = X[1000:11000,timestepsPerSampleWholeData - timestepsPerSample:]
    Y_mod = Y[1000:11000]
    timestepsPerSampleWholeData = X_mod.shape[1]
    # print(X_mod.shape)
    # print(Y_mod.shape)
    # print(X_mod[:1])

    # Splitting
    x_train, x_test, Y_train, Y_test = train_test_split(X_mod,Y_mod, test_size=0.15, shuffle=False)
    y_train = Y_train[:, 0:2]
    y_test = Y_test[:, 0:2]
    # print(y_train[:2])
    # print(y_test[:2])

    # Summarize
    num_samples = x_train.shape[0]
    test_samples = x_test.shape[0]
    print(f"Train shape: {x_train.shape}")
    print(f"Timesteps: {timestepsPerSampleWholeData}")
    print(f"Num Samples: {num_samples}")
    print(f"Test Samples: {test_samples}")
    print(f"Num features: {num_features}")
    return x_train, x_test, y_train, y_test, Y_test

In [26]:
# fit lstm model
def fit_lstmModel(x_train, y_train, x_test, y_test, batch_size, nb_epoch, neurons):
    LstmLayer = LSTM(
    units=neurons,
    activation="tanh",
    recurrent_activation="sigmoid",
    use_bias=True, # true if cuDNN
    kernel_initializer="glorot_uniform",
    recurrent_initializer="orthogonal",
    bias_initializer="zeros",
    unit_forget_bias=True,
    kernel_regularizer=None,
    recurrent_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    recurrent_constraint=None,
    bias_constraint=None,
    dropout=0.0, # !important parameter for optimization => 0 if cuDNN
    recurrent_dropout=0.0,
    seed=None,
    return_sequences=False,
    return_state=False,
    go_backwards=False,
    stateful=False,
    unroll=False, # false if cuDNN
    input_shape=(x_train.shape[1],x_train.shape[2]),
    # input_dim=(x_train.shape[1])
    )

    num_samples = x_train.shape[0]
    STEPS_PER_EPOCH = num_samples/batch_size

    model = Sequential() # initializing model
    # input layer and LSTM layer with 50 neurons
    model.add(LstmLayer)
    # model.add(Dense(100, activation='relu'))
    # model.add(Dense(100, activation='relu'))
    # model.add(Dense(20, activation='relu'))
    # outpute layer with sigmoid activation
    model.add(Dense(y_train.shape[1], activation='sigmoid'))

    # lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.001,
    decay_steps=STEPS_PER_EPOCH*100,
    decay_rate=0.9,
    staircase=False)

    def get_optimizer():
         # return tf.keras.optimizers.Adam(learning_rate=0.001)
        return tf.keras.optimizers.Adam(learning_rate= lr_schedule) # type: ignore

    optimizer = get_optimizer()

    # defining loss function, optimizer, metrics and then compiling model
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

    # fit model
    history = model.fit(x_train, y_train, epochs=nb_epoch, batch_size=batch_size, 
                        shuffle=False, validation_data=(x_test, y_test), verbose=1) # type: ignore
    return model, history

In [27]:
# Profit
def funcProfit(predict, Y_test):
    predict_classes = np.where(predict > 0.5, 1,0)
    concat = np.hstack((predict_classes, Y_test[:,2:]))
    df = pd.DataFrame(concat, columns=['Sell', 'Buy', 'Price'])

    # Absolute difference prices
    sum = 0
    for i in range(0, len(df)):
        if (i-1) > 0:
            diff = abs((df.at[i,'Price'] - df.at[i-1,'Price']))
            if diff < 0.5:
                if df.at[i,'Sell'] == 1:
                    sum += (df.at[i,'Price'] - df.at[i-1,'Price'])*(-1)
                elif df.at[i,'Buy'] == 1:
                    sum += (df.at[i,'Price'] - df.at[i-1,'Price'])
                if df.at[i,'Sell'] != df.at[i-1,'Sell']:
                    sum -= 0.03
    return sum

In [28]:
# Experiment
def experiment(repeats, epochs, neurons):
    # Data gen
    x_train, x_test, y_train, y_test, Y_test = data()

    # Evaluate declaration
    accuracy = list()
    profit = list()
    metrics = list()

    # Repeats
    for r in range(repeats):
        batch_size = 32
        # model
        model, history = fit_lstmModel(x_train, y_train, x_test, y_test, batch_size, epochs, neurons)

        # forecast the entire training dataset to build up state for forecasting
        predict = model.predict(x_test, batch_size=batch_size)
        test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0) # type: ignore

        metric = pd.DataFrame(history.history)
        metric['epoch'] = history.epoch

        metrics.append(metric)
        accuracy.append(test_acc*100)
        profit.append(funcProfit(predict, Y_test))
    
    return accuracy, profit, metrics

In [ ]:
# Run
from numpy import dtype

repeats = 3
results_acc = DataFrame(dtype='float')
results_profit = DataFrame(dtype='float')
metrics = list()

# vary training epochs
neurons = [50, 100, 200, 500, 1000]
epochs = 250
for n in neurons:
 results_acc[str(n)], results_profit[str(n)], metrics = experiment(repeats, epochs, n)

# summarize results
print(results_acc.describe())
print(results_profit.describe())

Train shape: (8500, 20, 4)
Timesteps: 20
Num Samples: 8500
Test Samples: 1500
Num features: 4
Epoch 1/250


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 20/266 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3255 - loss: 0.7146

2024-06-10 19:36:51.612612: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


266/266 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5674 - loss: 0.6735 - val_accuracy: 0.6740 - val_loss: 0.6228
Epoch 2/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6554 - loss: 0.6275 - val_accuracy: 0.6853 - val_loss: 0.5981
Epoch 3/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6652 - loss: 0.6171 - val_accuracy: 0.6920 - val_loss: 0.5908
Epoch 4/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6708 - loss: 0.6113 - val_accuracy: 0.6927 - val_loss: 0.5847
Epoch 5/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6769 - loss: 0.6059 - val_accuracy: 0.6980 - val_loss: 0.5782
Epoch 6/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6851 - loss: 0.6007 - val_accuracy: 0.7140 - val_loss: 0.5714
Epoch 7/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6909 - loss: 0.5957 - val_accuracy: 0.7207 - val_loss: 0.5661
Epoch 8/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6956 - loss: 0.5906 - val_accuracy: 0.7173

...

Epoch 1/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5956 - loss: 0.6874 - val_accuracy: 0.6787 - val_loss: 0.6170
Epoch 2/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6690 - loss: 0.6281 - val_accuracy: 0.6973 - val_loss: 0.5838
Epoch 3/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6858 - loss: 0.6006 - val_accuracy: 0.7067 - val_loss: 0.5751
Epoch 4/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6919 - loss: 0.5946 - val_accuracy: 0.7060 - val_loss: 0.5724
Epoch 5/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6927 - loss: 0.5895 - val_accuracy: 0.7173 - val_loss: 0.5667
Epoch 6/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6926 - loss: 0.5987 - val_accuracy: 0.7180 - val_loss: 0.5584
Epoch 7/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6900 - loss: 0.5864 - val_accuracy: 0.7193 - val_loss: 0.5544
Epoch 8/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6952 - loss: 0.5810 - val_accu

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5818 - loss: 0.7123 - val_accuracy: 0.5447 - val_loss: 0.7141
Epoch 2/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6095 - loss: 0.6781 - val_accuracy: 0.6940 - val_loss: 0.5977
Epoch 3/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6748 - loss: 0.6178 - val_accuracy: 0.7007 - val_loss: 0.5854
Epoch 4/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6830 - loss: 0.6026 - val_accuracy: 0.7047 - val_loss: 0.5753
Epoch 5/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6929 - loss: 0.5939 - val_accuracy: 0.7107 - val_loss: 0.5699
Epoch 6/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6911 - loss: 0.5893 - val_accuracy: 0.7200 - val_loss: 0.5621
Epoch 7/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6950 - loss: 0.5850 - val_accuracy: 0.7193 - val_loss: 0.5573
Epoch 8/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6959 - loss: 0.5835 - val_accu

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5958 - loss: 0.7243 - val_accuracy: 0.6780 - val_loss: 0.6153
Epoch 2/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6652 - loss: 0.6283 - val_accuracy: 0.7020 - val_loss: 0.5900
Epoch 3/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6803 - loss: 0.6114 - val_accuracy: 0.7000 - val_loss: 0.5780
Epoch 4/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6850 - loss: 0.6134 - val_accuracy: 0.4307 - val_loss: 0.8108
Epoch 5/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4879 - loss: 0.7399 - val_accuracy: 0.4307 - val_loss: 0.7845
Epoch 6/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5155 - loss: 0.7087 - val_accuracy: 0.4307 - val_loss: 0.7587
Epoch 7/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5495 - loss: 0.7014 - val_accuracy: 0.4313 - val_loss: 0.7849
Epoch 8/250
266/266 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5340 - loss: 0.7501 - val_accu

In [ ]:
# summarize results
print(results_acc.describe())
print(results_profit.describe())

In [ ]:
# Output

fig = make_subplots(rows=1, cols=2, subplot_titles=('Accuracy', 'Loss'))
for idx, m in enumerate(metrics):
    fig.add_trace(go.Scatter(x=m['epoch'], y=m['accuracy'], name=f'accuracy{str(idx)}', line_color='#0000ff', showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=m['epoch'], y=m['loss'], name=f'loss{str(idx)}', line_color='#0000ff', showlegend=False), row=1, col=2)
    fig.add_trace(go.Scatter(x=m['epoch'], y=m['val_accuracy'], name=f'val_accuracy{str(idx)}', line_color='#EF8260', showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=m['epoch'], y=m['val_loss'], name=f'val_loss{str(idx)}', line_color='#EF8260', showlegend=False), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy')
# fig.update_layout(width=1000, title='Accuracy and Loss')
fig.write_image(file=f"compare_models_{str(epochs)}.jpg", engine="kaleido")

In [ ]:
%matplotlib inline
print(results_acc.describe())

# 2 subplots in one row
fig, ax = plt.subplots(1, 2, figsize=(10, 6))

# Generate boxplots
results_acc.boxplot(ax=ax[0])
results_profit.boxplot(ax=ax[1])

# Set labels and titles
ax[0].set_title('Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[1].set_title('Profit')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Profit')

# Save to .png and show plot
plt.savefig(f'boxplot_{str(epochs)}.png')
plt.show()